# Template Network Definitions

<i>Version 1</i>

## References

1. ["Maintaining Knowledge about Temporal Intervals" by James F. Allen](https://cse.unl.edu/~choueiry/Documents/Allen-CACM1983.pdf) - Allen's original paper (PDF)
1. [Allen's Interval Algebra](https://www.ics.uci.edu/~alspaugh/cls/shr/allen.html) or [here](https://thomasalspaugh.org/pub/fnd/allen.html) - summarizes Allen's algebra of proper time intervals
1. [W3C Time Ontology in OWL](https://www.w3.org/TR/owl-time/) - temporal vocabulary used here is based on the W3C vocabulary of time
1. [bitsets Python package](https://bitsets.readthedocs.io/en/stable/) - used to implement Algebra relation sets and operations
1. [NetworkX Python package](http://networkx.github.io/) - used to represent directed graph of constraints
1. [Python format string syntax](https://docs.python.org/3/library/string.html#format-string-syntax) - used in Algebra summary method
1. [Spatial Ontology](https://www.w3.org/2017/sdwig/bp/) - I'm still looking for a standard spatial vocabulary; maybe start here
1. [Qualitative Spatial Relations (QSR) Library](https://qsrlib.readthedocs.io/en/latest/index.html) - an alternative library to the one defined here

## Dependencies

In [1]:
import os
import qualreas as qr

In [2]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')

In [10]:
pt_alg = qr.Algebra(os.path.join(path, "Algebras/LinearPointAlgebra.json"))
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/RightBranchingPointAlgebra.json"))
#pt_alg = qr.Algebra(os.path.join(path, "Algebras/LeftBranchingPointAlgebra.json"))

## Create Template 4-Point Network

In [11]:
def four_point_network(alg, lessthan_symbol, verbose=False):
    net = qr.Network(alg, "Four Point Network")
    lessthan = alg.relset(lessthan_symbol)
    # Start & End Points of Interval 1
    start1 = qr.TemporalEntity(["Point"], name="StartPt1")
    end1   = qr.TemporalEntity(["Point"], name=  "EndPt1")
    # Start & End Points of Interval 2
    start2 = qr.TemporalEntity(["Point"], name="StartPt2")
    end2   = qr.TemporalEntity(["Point"], name=  "EndPt2")
    net.add_constraint(start1, end1, lessthan, verbose)
    net.add_constraint(start2, end2, lessthan, verbose)
    return net, start1, end1, start2, end2

In [12]:
net4pt, s1, e1, s2, e2 = four_point_network(pt_alg, "<|=")

In [13]:
net4pt.summary()


Four Point Network: 4 nodes, 8 edges
  Algebra: LinearPointAlgebra
  StartPt1:
    => StartPt1: =
    => EndPt1: <|=
  EndPt1:
    => EndPt1: =
    => StartPt1: =|>
  StartPt2:
    => StartPt2: =
    => EndPt2: <|=
  EndPt2:
    => EndPt2: =
    => StartPt2: =|>


In [14]:
count = 0
consistent_nets = dict()
# Viewing the network as a matrix, 'elem13', refers to the element
# in row 1, col 3, and so on for 'elem23', etc.
for elem13 in pt_alg.elements:
    for elem23 in pt_alg.elements:
        for elem14 in pt_alg.elements:
            for elem24 in pt_alg.elements:
                net, pt1, pt2, pt3, pt4 = four_point_network(pt_alg, "<|=")
                rs13 = pt_alg.relset(elem13)
                rs23 = pt_alg.relset(elem23)
                rs14 = pt_alg.relset(elem14)
                rs24 = pt_alg.relset(elem24)
                net.add_constraint(pt1, pt3, rs13)
                net.add_constraint(pt2, pt3, rs23)
                net.add_constraint(pt1, pt4, rs14)
                net.add_constraint(pt2, pt4, rs24)
                if net.propagate():
                    count += 1
                    elem_key = ",".join([str(rs13), str(rs14), str(rs23), str(rs24)])
                    print("=========")
                    print(f"{rs13}, {rs14}")
                    print(f"{rs23}, {rs24}")
                    print("-----")
                    rs31 = net.edges[pt3, pt1]['constraint']
                    rs32 = net.edges[pt3, pt2]['constraint']
                    rs41 = net.edges[pt4, pt1]['constraint']
                    rs42 = net.edges[pt4, pt2]['constraint']
                    conv_key = ",".join([str(rs31), str(rs32), str(rs41), str(rs42)])
                    consistent_nets[elem_key] = (conv_key, net)
                    print(f"{rs31}, {rs32}")
                    print(f"{rs41}, {rs42}")
print(f"\n{count} consistent networks")

<, <
<, <
-----
>, >
>, >
<, <
=, <
-----
>, =
>, >
<, <
=, =
-----
>, =
>, =
<, <
>, <
-----
>, <
>, >
<, <
>, =
-----
>, <
>, =
<, <
>, >
-----
>, <
>, <
=, <
=, <
-----
=, =
>, >
=, =
=, =
-----
=, =
=, =
=, <
>, <
-----
=, <
>, >
=, <
>, =
-----
=, <
>, =
=, <
>, >
-----
=, <
>, <
=, =
>, >
-----
=, <
=, <
>, <
>, <
-----
<, <
>, >
>, <
>, =
-----
<, <
>, =
>, <
>, >
-----
<, <
>, <
>, =
>, =
-----
<, <
=, =
>, =
>, >
-----
<, <
=, <
>, >
>, >
-----
<, <
<, <

18 consistent networks


In [15]:
for key in consistent_nets:
    print(f"{key} : {consistent_nets[key]}")

<,<,<,< : ('>,>,>,>', <qualreas.Network object at 0x7ff26094b4a8>)
<,<,=,< : ('>,=,>,>', <qualreas.Network object at 0x7ff2307fadd8>)
<,<,=,= : ('>,=,>,=', <qualreas.Network object at 0x7ff260965b00>)
<,<,>,< : ('>,<,>,>', <qualreas.Network object at 0x7ff230f4eef0>)
<,<,>,= : ('>,<,>,=', <qualreas.Network object at 0x7ff230f55a58>)
<,<,>,> : ('>,<,>,<', <qualreas.Network object at 0x7ff230f58160>)
=,<,=,< : ('=,=,>,>', <qualreas.Network object at 0x7ff230f58eb8>)
=,=,=,= : ('=,=,=,=', <qualreas.Network object at 0x7ff230f5df60>)
=,<,>,< : ('=,<,>,>', <qualreas.Network object at 0x7ff230f5f9e8>)
=,<,>,= : ('=,<,>,=', <qualreas.Network object at 0x7ff230f62048>)
=,<,>,> : ('=,<,>,<', <qualreas.Network object at 0x7ff230f62710>)
=,=,>,> : ('=,<,=,<', <qualreas.Network object at 0x7ff230f62f28>)
>,<,>,< : ('<,<,>,>', <qualreas.Network object at 0x7ff230f68f28>)
>,<,>,= : ('<,<,>,=', <qualreas.Network object at 0x7ff230f6f048>)
>,<,>,> : ('<,<,>,<', <qualreas.Network object at 0x7ff230f6f7

In [23]:
key_name_mapping = {
    '<,<,<,<': 'B', '>,>,>,>': 'BI',
    '>,<,>,<': 'D', '<,<,>,>': 'DI',
    '=,<,>,=': 'E', '=,=,=,=': 'PE',
    '>,<,>,=': 'F', '<,<,>,=': 'FI',
    '<,<,=,<': 'M', '>,=,>,>': 'MI',
    '<,<,>,<': 'O', '>,<,>,>': 'OI',
    '=,<,>,<': 'S', '=,<,>,>': 'SI',
    '>,=,>,=': 'PF', '<,<,=,=': 'PFI',
    '=,<,=,<': 'PS', '=,=,>,>': 'PSI'
}

In [24]:
for key in consistent_nets:
    print(f"{key_name_mapping[key]} : {key} : {repr(consistent_nets[key][1])}")

B : <,<,<,< : <qualreas.Network object at 0x7ff26094b4a8>
M : <,<,=,< : <qualreas.Network object at 0x7ff2307fadd8>
PFI : <,<,=,= : <qualreas.Network object at 0x7ff260965b00>
O : <,<,>,< : <qualreas.Network object at 0x7ff230f4eef0>
FI : <,<,>,= : <qualreas.Network object at 0x7ff230f55a58>
DI : <,<,>,> : <qualreas.Network object at 0x7ff230f58160>
PS : =,<,=,< : <qualreas.Network object at 0x7ff230f58eb8>
PE : =,=,=,= : <qualreas.Network object at 0x7ff230f5df60>
S : =,<,>,< : <qualreas.Network object at 0x7ff230f5f9e8>
E : =,<,>,= : <qualreas.Network object at 0x7ff230f62048>
SI : =,<,>,> : <qualreas.Network object at 0x7ff230f62710>
PSI : =,=,>,> : <qualreas.Network object at 0x7ff230f62f28>
D : >,<,>,< : <qualreas.Network object at 0x7ff230f68f28>
F : >,<,>,= : <qualreas.Network object at 0x7ff230f6f048>
OI : >,<,>,> : <qualreas.Network object at 0x7ff230f6f710>
PF : >,=,>,= : <qualreas.Network object at 0x7ff230f6ff60>
MI : >,=,>,> : <qualreas.Network object at 0x7ff230f76080>
BI